<a href="https://colab.research.google.com/github/washaq00/TeethOrBraces/blob/master/Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")

PyTorch version: 2.1.0+cu121
torchvision version: 0.16.0+cu121


In [ ]:
import random
import os
import torchvision.transforms as transforms


train_dir = os.path.join(os.getcwd(),"/content/drive/MyDrive/data/train")
    # print(find_classes(train_dir))
test_dir = os.path.join(os.getcwd(),"/content/drive/MyDrive/data/test")

batch_size = 32



train_transform = transforms.Compose([
                        transforms.Resize(size=(224, 224)),
                        transforms.TrivialAugmentWide(num_magnitude_bins= 31),
                        transforms.ToTensor()])




In [ ]:
import pathlib
import os
from PIL import Image
from torch.utils.data import Dataset
from typing import Tuple, Dict, List #typical list is not



def find_classes(dir: str) -> Tuple[List[str],Dict[str, int]]:
    #get the class names with scanning directory (it checkes if there are any subfolder)
    classes = sorted(entry.name for entry in os.scandir(dir) if entry.is_dir())

    #show the error
    if not classes:
        raise FileNotFoundError(f"Couldn't find any classes")

    #creating a Dict with name and id
    classes_to_id = {cls_name: i for i, cls_name in enumerate(classes)}
    return classes, classes_to_id


class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform =None):
        self.classes, self.classes_idx = find_classes(img_dir)
        self.paths = list(pathlib.Path(img_dir).glob("*/*.png"))
        self.transform = transform

    def load_image(self, index: int) -> Image.Image:
        image_path = self.paths[index]
        return Image.open(image_path)

    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx):
        image = self.load_image(idx)
        class_name = self.paths[idx].parent.name
        class_idx = self.classes_idx[class_name]
        if self.transform:
            image = self.transform(image)
        return image, class_idx

In [ ]:
train_data = CustomImageDataset(img_dir=train_dir,  # target folder of images
                                    transform=train_transform)  # transforms to perform on labels (if necessary)

test_transform = transforms.Compose([
        transforms.Resize(size=(224, 224)),
        transforms.ToTensor()])

test_data = CustomImageDataset(img_dir=test_dir,
                                   transform=test_transform)

In [ ]:
from torch.utils.data import DataLoader
NUM_WORKERS = 1
class_names = train_data.classes
    # print(class_names)
    # img, label = train_data[0][0], train_data[0][1]
    # print_image(img)

train_dataloader = DataLoader(dataset=train_data,
                                  batch_size=batch_size,
                                  num_workers=NUM_WORKERS,
                                  shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                                 batch_size=batch_size,
                                 num_workers=NUM_WORKERS,
                                 shuffle=False)

In [ ]:
from torch import nn
VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ],
    "VGG19": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        512,
        "M",
    ],
}

class VGG(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(VGG, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_types["VGG16"])

        self.fcs = nn.Sequential(
            nn.Linear(512*8*8, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]
                in_channels = x
            elif x == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)

In [ ]:
import matplotlib.pyplot as plt
import torch
from time import perf_counter
from tqdm.auto import tqdm #progress bar
from torch.utils.data import DataLoader
def model_eval(model: torch.nn.Module,
               test_data: torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               acc1):
    torch.manual_seed(42)

    loss, accuracy = 0, 0
    model.eval()
    with torch.inference_mode():
        for x, y in tqdm(test_data):
            test_logits = model(x)
            loss += loss_fn(test_logits, y)
            accuracy += acc1(y_true=y,y_pred= test_logits.argmax(dim=1))

        loss /= len(test_data)
        accuracy /= len(test_data)

    return {"model_name": model.__class__.__name__,
            "model_loss": loss.item(), #returns the value of tensor
            "model_acc": accuracy}

def train_step(model: torch.nn.Module,
               train_data: torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimize,
               acc):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    tloss:float = 0
    tacc:float = 0
    model.train()

    for batch, (x, y) in enumerate(train_data):
        x, y = x.to(device), y.to(device)
        y_logits = model(x)
        loss = loss_fn(y_logits, y)
        optimize.zero_grad()
        loss.backward()
        optimize.step()
        model.eval()

        tloss += loss.item()
        tacc += acc(y_true=y, y_pred=y_logits.argmax(dim=1))
    tloss /= len(train_data)
    tacc /= len(train_data)

    return tloss, tacc


def test_step(model: torch.nn.Module,
              test_data: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              acc):
    tloss: float = 0
    tacc: float = 0
    model.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"

    for x, y in test_data:
        with torch.inference_mode():
            x, y = x.to(device), y.to(device)
            test_logits = model(x)
            loss = loss_fn(test_logits, y)

            tloss += loss.item()
            tacc += acc(y_true=y,y_pred=test_logits.argmax(dim=1))
    tloss /= len(test_data)
    tacc /= len(test_data)

    return tloss, tacc


def training_loop(model: torch.nn.Module,
                  train: torch.utils.data.DataLoader,
                  test: torch.utils.data.DataLoader,
                  optimizer,
                  loss_fn,
                  acc,
                  epochs: int = 5):

    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []}

    for epoch in tqdm(range(epochs)):

        train_loss, train_acc = train_step(model=model,
                                           train_data = train,
                                           optimize= optimizer,
                                           loss_fn = loss_fn,
                                           acc = acc)

        test_loss, test_acc = test_step(model=model,
                                        test_data=test,
                                        loss_fn=loss_fn,
                                        acc=acc)

        print(f"Epoch: {epoch} train loss: {train_loss}, train acc: {train_acc}, test loss: {test_loss}, test acc: {test_acc}")

        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)
        results["train_acc"].append(train_acc)
        results["test_acc"].append(test_acc)

    return results

In [ ]:
def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc



In [ ]:
import torch
from torch import nn


class TinyVgg(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)  # default stride value is same as kernel_size
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units * 16*16,
                      out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        # print(x.shape)
        x = self.block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x

In [ ]:
import torch
import torch.nn as nn


class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample=None, stride=1):
        super().__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels,
                               out_channels,
                               kernel_size=1,
                               stride=1,
                               padding=0,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels,
                               out_channels,
                               kernel_size=3,
                               stride=stride,
                               padding=1,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels,
                               out_channels*self.expansion,
                               kernel_size=1,
                               stride=1,
                               padding=0,
                               bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)


        if self.downsample is not None:
            identity = self.downsample(identity)

        x += identity
        x = self.relu(x)

        return x

In [ ]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import numpy as np
import torchvision
import math

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3,bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        #ResNet Layers
        self.layer1 = self._make_layer(block, layers[0], out_channels=64, stride=1)
        self.layer2 = self._make_layer(block, layers[1], out_channels=128, stride=2)
        self.layer3 = self._make_layer(block, layers[2], out_channels=256, stride=2)
        self.layer4 = self._make_layer(block, layers[3], out_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*4, num_classes) #fully conected layer

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, out_channels, stride):
        downsample = None
        layers = []

        if stride != 1 or self.in_channels != out_channels * 4:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels,
                          out_channels * 4,
                          kernel_size=1,
                          stride=stride,
                          bias=False),
                nn.BatchNorm2d(out_channels * 4))

        layers.append(block(self.in_channels, out_channels, downsample, stride))
        self.in_channels = out_channels*4

        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

def ResNet50(pretrained=True, img_channels=3, num_classes=2):

    model = ResNet(ResNetBlock, [3,4,6,3], img_channels, num_classes)

    if pretrained:
        pretrained_dict = model_zoo.load_url(model_urls['resnet50'])
        model_dict = model.state_dict()
        pretrained_dict = {key.replace("module.", ""): value for key, value in pretrained_dict.items()}
        model_dict.update(pretrained_dict)
        model.load_state_dict(pretrained_dict)
    return model


In [ ]:
import torch.nn.functional as F
class Net(nn.Module):
    # conv2d_depths = [1, 2, 4, 8, 16, 32]
    conv2d_depths = [3, 16, 32, 64, 128, 128]
    linear_features = [2048, 1024, 2]

    def __init__(self):
        super(Net, self).__init__()
        # convolutional layers
        self.conv1 = nn.Conv2d(self.conv2d_depths[0], self.conv2d_depths[1], 3, padding=1)  # 224
        self.conv2 = nn.Conv2d(self.conv2d_depths[1], self.conv2d_depths[2], 3, padding=1)  # 112
        self.conv3 = nn.Conv2d(self.conv2d_depths[2], self.conv2d_depths[3], 3, padding=1)  # 56
        self.conv4 = nn.Conv2d(self.conv2d_depths[3], self.conv2d_depths[4], 3, padding=1)  # 28
        self.conv5 = nn.Conv2d(self.conv2d_depths[4], self.conv2d_depths[5], 3, padding=1)  # 14
        # conv2d dropout
        self.conv2_dropout = nn.Dropout2d(0.2)
        # pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # fully connected layers
        self.in_features = self.conv2d_depths[-1] * 14 ** 2
        self.linear1 = nn.Linear(self.in_features, self.linear_features[0])
        self.linear2 = nn.Linear(self.linear_features[0], self.linear_features[1])
        self.linear3 = nn.Linear(self.linear_features[1], self.linear_features[2])
        # dropout
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # Define forward behavior
        x = F.relu(self.conv1(x))
        x = self.conv2_dropout(x)
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.conv2_dropout(x)
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.conv2_dropout(x)
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.conv2_dropout(x)
        x = self.pool(x)
        x = F.relu(self.conv5(x))
        x = self.conv2_dropout(x)

        # flatten, otherwise it won't work
        x = x.view(-1, self.in_features)


        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = F.relu(self.linear2(x))
        x = self.dropout(x)
        x = F.relu(self.linear3(x))
        x = self.dropout(x)

        return x

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
from torchvision import models

vgg = models.vgg16().to(device)
# model_0 = ResNet50(num_classes=len(train_data.classes)).to(device)
model_1 = Net().to(device)
model_0= TinyVgg(input_shape = 3,
                 hidden_units = 10,
                 output_shape = 2).to(device)


optimizer = torch.optim.SGD(params=model_1.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path, from_epoch=0, valid_loss_min=np.Inf):
    """returns trained model"""

    for epoch in range(1, n_epochs + 1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            """ From https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html and
             https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
            """
            optimizer.zero_grad()
            if isinstance(model, models.Inception3):
                output, aux_output = model(data)
                loss1 = criterion(output, target)
                loss2 = criterion(aux_output, target)
                loss = loss1 + 0.4*loss2
            else:
                output = model(data)
                loss = criterion(output, target)

            loss.backward()
            optimizer.step()
            train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))

        ######################
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model.forward(data)
            loss = criterion(output, target)
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

        # print training/validation statistics
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch + from_epoch,
            train_loss,
            valid_loss
        ))

        # save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}).  Saving model ...')
            try:
                save_model(model, save_path)
            except:
                print('Could not save the model')
            valid_loss_min = valid_loss

    # return trained model
    return model

In [ ]:
model_0_results = training_loop(model=model_1,
                                    train=train_dataloader,
                                    test=test_dataloader,
                                    optimizer=optimizer,
                                    loss_fn=loss_fn,
                                    acc=accuracy_fn,
                                    epochs=100)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0 train loss: 0.6933111357934696, train acc: 46.13530927835052, test loss: 0.6931474757194519, test acc: 54.0
Epoch: 1 train loss: 0.6931472691064028, train acc: 55.975515463917525, test loss: 0.6931474661827087, test acc: 54.5
Epoch: 2 train loss: 0.6931467928837255, train acc: 55.67654639175258, test loss: 0.6931473088264465, test acc: 54.5
Epoch: 3 train loss: 0.69314790693755, train acc: 55.920103092783506, test loss: 0.6931472969055176, test acc: 54.625
Epoch: 4 train loss: 0.6931524307457442, train acc: 55.878865979381445, test loss: 0.6931472969055176, test acc: 54.625
Epoch: 5 train loss: 0.693150276990281, train acc: 55.81056701030928, test loss: 0.6931472969055176, test acc: 54.625
Epoch: 6 train loss: 0.6931494142591339, train acc: 55.81958762886598, test loss: 0.6931472969055176, test acc: 54.625
Epoch: 7 train loss: 0.6931484489096809, train acc: 55.80541237113402, test loss: 0.6931472969055176, test acc: 54.625
Epoch: 8 train loss: 0.6931485232618666, train acc: 55